In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import subprocess

import libs.gen_selection as gs
import libs.graphical_interface as gui
import libs.utility as ut
import libs.machine_learning as ml
import libs.taylor as tr

In [3]:
#stratData = ut.readData("strat_data", "static_pred")
stratData = ut.readData("strat_data", "dynamic_pred")
display(stratData)

,Aj,Bj,Aa,Ba
0,-86.578340,37.492741,-49.337102,18.032950
1,-84.356677,28.402022,-47.323965,9.245189
2,-68.680094,-26.467341,-63.166301,38.666377
3,-51.816150,-40.250791,-69.296155,-37.647572
4,-101.934827,26.035904,-48.801369,-39.777466
...,...,...,...,...
95,-80.509245,52.792832,-61.216507,49.770003
96,-73.144517,-29.256973,-74.352571,3.868980
97,-17.672331,-16.054416,-62.191732,-12.702416
98,-65.745168,-27.598721,-79.112739,-50.851873


In [17]:
#mpData = ut.readData("mp_data", "static_pred")
mpData = ut.readData("mp_data", "dynamic_pred")
with pd.option_context('display.precision', 4, 'display.max_columns', 10):
    display(mpData)

,M1,M2,M3,M4,M5,...,M6M7,M6M8,M7M7,M7M8,M8M8
0,74.7903,-86.6016,-27747.5167,-746.1274,126.9281,...,7.6780e+05,131912.5266,4.1203e+07,7.0789e+06,1.2162e+06
1,77.9007,-83.8132,-15923.1237,-422.3181,129.7464,...,1.9699e+05,129655.4831,2.8466e+06,1.8735e+06,1.2331e+06
2,99.8479,-69.7035,-13827.7135,-478.4003,107.5672,...,3.4057e+06,118968.1861,8.7095e+08,3.0424e+07,1.0628e+06
3,123.4574,-81.6701,-31980.0093,-1604.3925,98.9854,...,1.7417e+06,51251.8209,7.8272e+08,2.3032e+07,6.7773e+05
4,53.2912,-61.2998,-13380.5838,-820.0708,127.6781,...,2.6726e+06,150989.6912,9.7546e+08,5.5109e+07,3.1134e+06
...,...,...,...,...,...,...,...,...,...,...,...
95,83.2871,-103.0646,-55014.8157,-1393.8009,110.2969,...,6.0827e+06,197966.9624,2.3907e+09,7.7809e+07,2.5324e+06
96,93.5977,-62.6724,-16896.1801,-474.9829,91.9064,...,2.3963e+04,3193.4854,8.7306e+04,1.1635e+04,1.5506e+03
97,171.2587,-137.1606,-5087.6678,-4013.6105,108.9316,...,2.7310e+05,34111.6064,1.0144e+07,1.2670e+06,1.5825e+05
98,103.9568,-72.5466,-15035.1462,-584.0449,85.2422,...,2.1721e+06,55053.4391,2.6055e+09,6.6038e+07,1.6738e+06


In [20]:
stratFitData = ut.readData("strat_fit_data", "static_pred")
display(stratFitData.head(8))

,Aj,Bj,Aa,Ba,fit
0,-32.124539,8.978988,-74.281205,20.516454,-2.458198
1,-64.557149,-63.195235,-43.094090,39.179464,-0.929736
2,-49.664621,29.694500,-63.846178,-12.400792,-1.851534
3,-112.078487,-16.019598,-92.800682,24.854612,-0.491697
4,-79.946916,-2.191970,-91.023665,-43.527015,-0.897783
5,-66.366043,-39.541403,-63.703149,11.107243,-1.420374
6,-49.476338,1.366716,-67.027336,-30.347347,-1.735946
7,-99.813805,-29.069204,-79.910053,-23.382501,-0.580399


In [9]:
#selData = ut.readData("sel_data", "static_pred")
selData = ut.readData("sel_data", "dynamic_pred")
with pd.option_context('display.precision', 4, 'display.max_columns', 10):
    display(selData)

,class,M1,M2,M3,M4,...,M6M7,M6M8,M7M7,M7M8,M8M8
0,-1,-3.1103,-2.7884,-11824.3931,-323.8093,...,5.7081e+05,2257.0435,3.8356e+07,5.2053e+06,-1.6938e+04
1,1,3.1103,2.7884,11824.3931,323.8093,...,-5.7081e+05,-2257.0435,-3.8356e+07,-5.2053e+06,1.6938e+04
2,1,-25.0575,-16.8982,-13919.8032,-267.7270,...,-2.6379e+06,12944.3405,-8.2975e+08,-2.3345e+07,1.5339e+05
3,-1,25.0575,16.8982,13919.8032,267.7270,...,2.6379e+06,-12944.3405,8.2975e+08,2.3345e+07,-1.5339e+05
4,-1,-48.6671,-4.9315,4232.4926,858.2652,...,-9.7395e+05,80660.7057,-7.4152e+08,-1.5953e+07,5.3846e+05
...,...,...,...,...,...,...,...,...,...,...,...
8005,1,-67.3020,64.6140,-9947.4783,3429.5655,...,1.8990e+06,20941.8326,2.5953e+09,6.4771e+07,1.5155e+06
8006,-1,23.6707,-12.6306,-4874.0087,-1945.2220,...,-1.6731e+06,-15595.5389,-2.7457e+09,-6.9117e+07,-1.6394e+06
8007,1,-23.6707,12.6306,4874.0087,1945.2220,...,1.6731e+06,15595.5389,2.7457e+09,6.9117e+07,1.6394e+06
8008,-1,-43.6312,51.9834,-14821.4870,1484.3435,...,2.2587e+05,5346.2937,-1.5032e+08,-4.3462e+06,-1.2385e+05


In [11]:
#compareCoefData = ut.readData("compare_coef_data", "static_pred")
compareCoefData = ut.readData("compare_coef_data", "dynamic_pred")
display(compareCoefData[['machine', 'nearPnt', 'optPnt']])

,machine,nearPnt,optPnt
lam1,1.000000e+00,1.000000e+00,1.000000e+00
lam2,1.153898e+00,2.917442e+00,1.228372e-01
lam3,2.199009e-03,4.846154e-04,4.846154e-04
lam4,3.379638e-02,1.307692e-02,1.307692e-02
lam5,1.873764e+00,1.006207e+01,6.011430e+00
lam6,5.528861e+00,1.568545e+01,6.618185e+00
lam7,5.319998e-04,2.323860e-03,1.388354e-03
lam8,3.894318e-02,4.791463e-02,2.862586e-02
lam11,3.422594e-03,-2.374289e-03,-2.615318e-03
lam12,-2.744787e-03,-3.682866e-03,-1.893263e-04


In [10]:
stratPopData = ut.readData("strat_pop_data", "dynamic_pred").sort_values(by=['t'], ascending=False)
display(stratPopData[['Aj', 'Bj', 'Aa', 'Ba', 't']])

,Aj,Bj,Aa,Ba,t
99,-34.580000,-3.290000,-83.320000,-51.570000,5000.000000
84,-56.278235,-9.001956,-79.035845,-57.606978,537.105371
14,-28.532572,19.809711,-77.734577,-53.898837,522.355224
78,-44.794071,19.119975,-85.888024,-43.776393,506.305063
15,-30.568359,13.021526,-88.037199,-37.156769,492.804928
...,...,...,...,...,...
87,-107.999811,1.217076,-66.186475,63.206710,43.200432
86,-55.560500,-46.102990,-66.388019,48.856746,42.950430
89,-49.455138,45.621872,-53.504832,50.590479,42.600426
92,-74.969110,39.227673,-73.558576,52.191099,42.400424


In [3]:
comparePqrsData = ut.readData("compare_pqrs_data", "dynamic_pred")
display(comparePqrsData)

,p,q,r,s
true,1.565672,0.33125,2.555424,3.944674
restored,1.544959,0.39217,2.555291,3.873234


In [4]:
compareParamData = ut.readData("compare_param_data", "dynamic_pred")
display(compareParamData)

,a_j,b_j,g_j,d_j,a_a,b_a,g_a,d_a
true,0.013000,0.000006,0.002660,0.000170,0.042000,0.000010,0.106400,0.000200
restored,0.013558,0.000004,0.003149,0.000142,0.069851,0.000007,0.104473,0.000533


In [5]:
tmpData = pd.DataFrame({'Aj': [-34.58, -32.0], 'Bj': [-3.29, -8.0], 'Aa': [-83.32, -86.0], 'Ba': [-51.57, -54.0]}, index=['true', 'restored'])
display(tmpData)

,Aj,Bj,Aa,Ba
true,-34.58,-3.29,-83.32,-51.57
restored,-32.00,-8.00,-86.00,-54.00
